In [83]:

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack, csr_matrix
import ast
def recomendar_juegos_genero(item_id):
    """ Se consulta  un item_id, y se recomiendan 5 juegos basados 
    en su similitud respecto a sus generos respectivos"""
    try:
        # Cargar los dataframes
        df_steam =  pd.read_csv(r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\steam_post_etl.csv", sep=';', on_bad_lines='skip', usecols=['item_id', 'title', 'genres','specs' ]) 
        # df_reviews = pd.read_csv(r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\reviews_post_etl.csv", sep=';', usecols=['item_id', 'sentiment_analysis'])
       
        # Verificar si el item_id existe en el dataframe
        if item_id not in df_steam['item_id'].values:
            raise ValueError(f"El item_id {item_id} no está en la base de datos.")

        # Filtrar el dataframe de Steam para obtener solo los juegos relevantes
        df_steam = df_steam.dropna(subset=['genres'])  # Eliminar filas con NaN en géneros

        # Obtener el género del juego buscado
        game_genres_str = df_steam[df_steam['item_id'] == item_id]['genres'].values[0]
        game_genres = ast.literal_eval(game_genres_str)
        
        # verificar si el juego no tiene géneros válidos
        if not game_genres_str or game_genres_str == '[]':
            raise ValueError(f"El juego con item_id {item_id} no tiene géneros asignados, por lo que no podemos darte una recomendación para géneros similares.")


        # Filtro los juegos que tengan al menos alguno de esos generos y restablecer indices
        filtered_df_steam = df_steam[df_steam['genres'].apply(lambda x: any(genre in eval(x) for genre in game_genres))]
        filtered_df_steam = filtered_df_steam.reset_index(drop=True)

        # Si no hay suficientes juegos similares en el filtro
        if filtered_df_steam.shape[0] <= 1:
            raise ValueError(f"No se encontraron suficientes juegos similares al juego con item_id {item_id}.")

        # Vectorizar la columna 'genres':
        # Obj: Convertir el texto de la columna genres en una representación numérica que pueda ser utilizada para calcular similitudes.
        tfidf = TfidfVectorizer()
        tfidf_matrix = tfidf.fit_transform(filtered_df_steam['genres'])
        
         # Obtener el índice del juego buscado
        idx = filtered_df_steam.index[filtered_df_steam['item_id'] == item_id].tolist()[0]

         # Calcular la similitud coseno entre el juego buscado y todos los juegos
        sim_scores = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()
   
        #Calcular las puntuaciones de similitud entre el juego buscado y todos los demás juegos    
        sim_scores_idx = list(enumerate(sim_scores))

        # Ordenar las puntuaciones de similitud en orden descendente (juegos más similares primero)
        sim_scores_idx = sorted(sim_scores_idx, key=lambda x: x[1], reverse=True)

        # Obtener los índices de los juegos más similares (excluyendo el propio juego)
        # Esto se hace porque la primera entrada (sim_scores[0]) es el propio juego, que tendrá una similitud de 1  y no lo queremos en las recomendaciones.
        top_indices = [i[0] for i in sim_scores_idx[1:6]]

        # Retornar los títulos de los juegos recomendados
        return filtered_df_steam.iloc[top_indices][['title', 'genres']]
    except ValueError as e:
        return str(e)  # Retornar el mensaje de error si ocurre




In [84]:
recomendar_juegos_genero(2500)

,title,genres
10,Twisted Enhanced Edition,['Action']
19,Advent Rising,['Action']
24,Devil May Cry® 3 Special Edition,['Action']
26,Final DOOM,['Action']
28,Ultimate Doom,['Action']


In [94]:
def recomendar_juegos_genero_spec(item_id):
    # Cargar el dataframe
    df_steam = pd.read_csv(r"M:\Documentos\Mai\Henry\Cursado\P.I. 1\Bases de datos\Archivos Post ETL\steam_post_etl.csv", sep=';', on_bad_lines='skip', usecols=['item_id', 'title', 'genres', 'specs'])
    
    # Verificar si el item_id existe en el dataframe, sino error
    if item_id not in df_steam['item_id'].values:
        raise ValueError(f"El item_id {item_id} no está en la base de datos.")

    # Filtra el DataFrame para eliminar filas que tengan valores nulos en las columnas genres o specs, y reinicia los índices
    df_steam = df_steam.dropna(subset=['genres', 'specs']).reset_index(drop=True)

    # Selecciona la fila del DataFrame que corresponde al item_id buscado en la funcion
    game_row = df_steam[df_steam['item_id'] == item_id]

    # Verificar si se encontró el juego
    if game_row.empty:
        raise ValueError(f"No se encontró el juego con item_id {item_id}.")

    # Extraer los géneros y especificaciones del juego encontrado
    game_genres_str = game_row['genres'].values[0]
    game_specs_str = game_row['specs'].values[0]
    game_title = game_row['title'].values[0]
    
    #Convierte las cadenas de texto de game_genres_str y game_specs_str
    game_genres = ast.literal_eval(game_genres_str) if isinstance(game_genres_str, str) else game_genres_str
    game_specs = ast.literal_eval(game_specs_str) if isinstance(game_specs_str, str) else game_specs_str

    # Unir en una sola fila Genero + Specs
    genres_specs_item = list(set(game_specs + game_genres))

    # Definir una función interna para Convertir las columnas de strings en listas
    def combinar_generos_specs(row):
        
        genres = ast.literal_eval(row['genres']) if isinstance(row['genres'], str) else row['genres']
        specs = ast.literal_eval(row['specs']) if isinstance(row['specs'], str) else row['specs']
        
        # Unir ambos en una sola lista y eliminar duplicados si los hubiera
        return list(set(genres + specs))

    # Aplicar la función a cada fila del dataframe y crear la nueva columna 'genres_mas_specs'
    df_steam['genres_mas_specs'] = df_steam.apply(combinar_generos_specs, axis=1)

    # Convertir cada lista en 'genres_mas_specs_str' en una cadena de texto
    df_steam['genres_mas_specs_str'] = df_steam['genres_mas_specs'].apply(lambda x: ' '.join(x))

    # Vectorizar la columna 'genres_mas_specs_str'
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(df_steam['genres_mas_specs_str'])

    # Obtener el índice del juego buscado en el DataFrame
    idx = df_steam.index[df_steam['item_id'] == item_id].tolist()[0]

    # Calcular la similitud coseno entre el juego buscado y todos los juegos
    sim_scores = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()

    # Calcular las puntuaciones de similitud entre el juego buscado y todos los demás juegos    
    sim_scores_idx = list(enumerate(sim_scores))
    # Ordenar las puntuaciones de similitud en orden descendente (juegos más similares primero)
    sim_scores_idx = sorted(sim_scores_idx, key=lambda x: x[1], reverse=True)

    # Obtener los índices de los juegos más similares (excluyendo el propio juego)
    top_indices = [i[0] for i in sim_scores_idx[1:6]]

    # Crear la respuesta estructurada
    recomendaciones = df_steam.iloc[top_indices][['title', 'genres', 'specs']].to_dict(orient='records')
    
    respuesta = {
        'Juego Buscado': game_title,
        'Top 5 Recomendaciones': recomendaciones
    }
    
    return respuesta

In [95]:
recomendar_juegos_genero_spec(508600)

{'Juego Buscado': 'ROD: Revolt Of Defense',
 'Top 5 Recomendaciones': [{'title': 'Tactics: Bludgeons Blessing',
   'genres': "['Action', 'Simulation', 'Strategy']",
   'specs': "['Single-player', 'Steam Trading Cards']"},
  {'title': 'Mechs &amp; Mercs: Black Talons',
   'genres': "['Action', 'Indie', 'Simulation', 'Strategy']",
   'specs': "['Single-player', 'Steam Achievements', 'Steam Trading Cards']"},
  {'title': 'Xenonauts',
   'genres': "['Indie', 'Simulation', 'Strategy']",
   'specs': "['Single-player', 'Steam Trading Cards']"},
  {'title': 'Project Tarvotan',
   'genres': "['Indie', 'Simulation', 'Strategy']",
   'specs': "['Single-player', 'Steam Trading Cards']"},
  {'title': 'VilleTown',
   'genres': "['Indie', 'Simulation', 'Strategy']",
   'specs': "['Single-player', 'Steam Trading Cards']"}]}

In [97]:
recomendar_juegos_genero(508600)

,title,genres
1928,Kill The Bad Guy,"['Action', 'Indie', 'Simulation', 'Strategy']"
2213,GoD Factory: Wingmen,"['Action', 'Indie', 'Simulation', 'Strategy']"
2569,Mechs &amp; Mercs: Black Talons,"['Action', 'Indie', 'Simulation', 'Strategy']"
2677,Universal Combat CE,"['Action', 'Indie', 'Simulation', 'Strategy']"
2701,Evolution II: Fighting for Survival,"['Action', 'Indie', 'Simulation', 'Strategy']"
